# The Math Library - Semantic search for competition math

In [ ]:
from datasets import load_dataset
from txtai.embeddings import Embeddings

The search database will be [NuminaMath 1.5](https://huggingface.co/datasets/AI-MO/NuminaMath-1.5), which contains around 900k competition-level math problems and their solution.

In [2]:
ds = load_dataset("AI-MO/NuminaMath-1.5", "default")['train'] #.select(range(10**4))
print(ds[:5])

{'problem': ['\nProblem 1. Find all prime numbers $p$ for which there exist positive integers $x, y$ and $z$ such that the number\n\n$$\nx^{p}+y^{p}+z^{p}-x-y-z\n$$\n\nis a product of exactly three distinct prime numbers.\n', '\nProblem 2. Let $a, b$ be two distinct real numbers and let $c$ be a positive real number such that\n\n$$\na^{4}-2019 a=b^{4}-2019 b=c .\n$$\n\nProve that $-\\sqrt{c}<a b<0$.\n', '\nProblem 3. Triangle $A B C$ is such that $A B<A C$. The perpendicular bisector of side $B C$ intersects lines $A B$ and $A C$ at points $P$ and $Q$, respectively. Let $H$ be the orthocentre of triangle $A B C$, and let $M$ and $N$ be the midpoints of segments $B C$ and $P Q$, respectively. Prove that lines $H M$ and $A N$ meet on the circumcircle of $A B C$.\n', '\nProblem 4. A $5 \\times 100$ table is divided into 500 unit square cells, where $n$ of them are coloured black and the rest are coloured white. Two unit square cells are called adjacent if they share a common side. Each of

We also initialize our embedding database based on [nli-mpnet-base-v2](https://huggingface.co/sentence-transformers/nli-mpnet-base-v2), which is a 100M parameter model that maps sentences to a 768 dimensional vector.

In [3]:
embeddings = Embeddings({"path": "sentence-transformers/nli-mpnet-base-v2"})

Since the embedding process takes a few hours, we will save the results to give the option to simply load the embeddings from disk enabled with `load_from_disk = True`.

In [4]:
load_from_disk = True

if not load_from_disk:
    embeddings.index([(uid, {"text": row["problem"], "answer": row["solution"]}, None) for uid, row in enumerate(ds)])
    embeddings.save("numina-math.tar.gz")
else:
    embeddings.load("numina-math.tar.gz")

Now we can query for similar math problems using our embedding database, we test it with a simple keyword search:

In [5]:
# search top results, number specified by limit
def question(text, limit=5):
  result = embeddings.search(text, limit)

  for (uid, score) in result:
      print(score, ds[uid])

question("Cyclic quadrilateral")

0.7590136528015137 {'problem': 'A cyclic quadrilateral is divided into four quadrilaterals by two lines passing through its inner point. Three of these quadrilaterals are cyclic with equal circumradii. Prove that the fourth part also is cyclic quadrilateral and its circumradius is the same.\n\n(A.Blinkov) ', 'solution': None, 'answer': None, 'problem_type': 'Geometry', 'question_type': 'proof', 'problem_is_valid': 'Yes', 'solution_is_valid': 'Yes', 'source': 'aops_forum', 'synthetic': False}
0.7485953569412231 {'problem': '371. Cyclic Quadrilateral. Construct a cyclic quadrilateral* such that each of its sides touches one of four fixed circles.', 'solution': '371. There are infinitely many such quadrilaterals. Let the given circles be denoted by $C_{1}, C_{2}, C_{3}$, and $C_{4}$. Construct an arbitrary cyclic quadrilateral on the plane with sides $s_{1}, s_{2}, s_{3}$, and $s_{4}$. Then draw a line $t_{i}$ parallel to $s_{i}$ and tangent to $C_{i}, i=1,2,3,4$. The quadrilateral with s

We start with a problem 1 from the "Bundeswettbewerb Mathematik 2018 Round 2", and our top match is the exact solution:

In [13]:
question("Anja and Bernd alternately take stones from a pile initially containing $n$ stones ($n \geq 2$). Anja begins and, in her first move, removes at least one, but not all, of the stones. After that, the player whose turn it is removes at least one, but at most as many stones as were removed in the immediately preceding move. The player who takes the last stone wins. For which values of $n$ can Anja force a win, and for which values can Bernd force a win?", limit=1)

0.9464877843856812 {'problem': 'Anja and Bernd take turns in removing stones from a heap, initially consisting of $n$ stones ($n \\ge 2$). Anja begins, removing at least one but not all the stones. Afterwards, in each turn the player has to remove at least one stone and at most as many stones as removed in the preceding move. The player removing the last stone wins.\n\nDepending on the value of $n$, which player can ensure a win?', 'solution': 'To determine which player can ensure a win, we need to analyze the game based on the initial number of stones, \\( n \\). We will consider different cases based on the parity (even or odd) and specific properties of \\( n \\).\n\n1. **Case 1: \\( n \\) is even**\n   - Let \\( n = 2k \\).\n   - If \\( k \\) is odd, Anja can take 2 stones on her first move, leaving \\( n - 2 = 2k - 2 \\) stones.\n   - Now, \\( 2k - 2 \\) is even, and Bernd is forced to take at least 1 stone but not more than 2 stones.\n   - If Bernd takes 2 stones, the remaining s

For problem 631211 from the "Mathematik Olympiade" we find some close matches, which can help solve our problem:

In [14]:
question("For a natural number n, let P (n) be the product of its digits other than 0. For example, P (2023) = 2 * 2 * 3 = 12. Determine how many four-digit numbers n exist with the property P (n) = 12. Keep in mind that numbers can include the digit 0")

0.8537325263023376 {'problem': '11.2. Let $n$ be a natural number not ending in 0, and $R(n)$ be the four-digit number obtained from $n$ by reversing the order of its digits, for example $R(3257)=7523$: Find all natural four-digit numbers $n$ such that $R(n)=4n+3$.', 'solution': 'Answer: 1997.\n\nSolution. Consider the decimal representation of the original four-digit number $n=\\overline{a b c d}$, then $R(n)=\\overline{d c b a}=4 n+3$ is also a four-digit number. Therefore, $4 a \\leq 9$, so $a=1$ or $a=2$. Moreover, the number $R(n)=\\overline{d c b a}=4 n+3$ is odd and ends in $a$, hence $a=1$. If the number $4 n+3$ ends in 1, then $n$ must end in 2 or 7, that is, $d=2$ or $d=7$. The first is impossible, since $d \\geq 4 a \\geq 4$, so $d=7$. Substituting the found values into the equation from the condition: $R(n)=4 \\cdot \\overline{a b c d}+3=4031+400 b+40 c=7001+100 c+10 b$, from which $13 b-2 c=99$. From this, $b \\geq \\frac{99}{13}=7 \\frac{8}{13}$, that is, $b=8$ or $b=9$. 

The same is true for problem 631221 of the same competition:

In [15]:
question("Given are the odd integers a, b, c and d. If you sort them in ascending order, you get a sequence of four consecutive odd integers. They also fulfill the system of equations 3a * c = c * d , (1) b + c + d = a * c . (2) Determine c * d, b + c and a * c.", limit=10)

0.8614740371704102 {'problem': '12. Given four integers $a, b, c, d$ are all even, and $0<a<b<c<d, d-a=90$, if $a, b, c$ form an arithmetic sequence, and $b, c, d$ form a geometric sequence, then $a+b+c+d=$ $\\qquad$ .', 'solution': 'Let the four numbers be $b-m, b, b+m, \\frac{(b+m)^{2}}{b}$, then $\\frac{(b+m)^{2}}{b}-b+m=90$ $\\Rightarrow m(m+3 b)=90 b$. Since $b, m$ are both even, $m+3 b \\equiv m(\\bmod 6)$, thus $m, m+3 b$ are both multiples of 6. Also, $0<m<b \\Rightarrow 3 b<m+3 b<4 b \\Rightarrow \\frac{90}{4}<m<30 \\Rightarrow m=24$, hence $24(24+3 b)=90 b \\Rightarrow 8+b=\\frac{5}{4} b \\Rightarrow b=32$.\n\nTherefore, $a+b+c+d=8+32+56+98=194$.', 'answer': '194', 'problem_type': 'Algebra', 'question_type': 'math-word-problem', 'problem_is_valid': 'Yes', 'solution_is_valid': 'Yes', 'source': 'olympiads', 'synthetic': False}
0.8614612221717834 {'problem': '1. For four numbers $a, b, c, d$ it is satisfied: $d>c, a+b=c+d, a+d<b+c$. Arrange these four numbers in order of magnitu

Even on the quite hard problem 2 from "Bundeswettbewerb Mathematik 2018 Runde 2" we obtain a lot of useful inspiration:

In [18]:
question("Find a real function $f$ with the property $f(1 - f(x)) = x$ for all $x \in \mathbb{R}$.")

0.9482497572898865 {'problem': 'Find a real function $f : [0,\\infty)\\to \\mathbb R$ such that $f(2x+1) = 3f(x)+5$, for all $x$ in $[0,\\infty)$.', 'solution': "To find a real function \\( f : [0,\\infty) \\to \\mathbb{R} \\) such that \\( f(2x+1) = 3f(x) + 5 \\) for all \\( x \\in [0,\\infty) \\), we can proceed as follows:\n\n1. **Assume a form for \\( f(x) \\)**:\n   Let's assume \\( f(x) = c \\) where \\( c \\) is a constant. Then, substituting into the functional equation:\n   \\[\n   f(2x+1) = 3f(x) + 5\n   \\]\n   becomes:\n   \\[\n   c = 3c + 5\n   \\]\n\n2. **Solve for \\( c \\)**:\n   \\[\n   c = 3c + 5\n   \\]\n   \\[\n   c - 3c = 5\n   \\]\n   \\[\n   -2c = 5\n   \\]\n   \\[\n   c = -\\frac{5}{2}\n   \\]\n\n3. **Verify the solution**:\n   Substitute \\( f(x) = -\\frac{5}{2} \\) back into the original functional equation to verify:\n   \\[\n   f(2x+1) = -\\frac{5}{2}\n   \\]\n   \\[\n   3f(x) + 5 = 3\\left(-\\frac{5}{2}\\right) + 5 = -\\frac{15}{2} + 5 = -\\frac{15}{2} + \\

Lastly for problem 4 of the same competition we again find a perfect match including the solution:

In [20]:
question("Determine all natural numbers n > 1 such that the following holds: If each lattice point in a square grid in the plane is colored with one of n given colors, then there always exist three lattice points of the same color that form an isosceles right triangle, whose legs are parallel to the grid lines.")

0.9152128100395203 {'problem': 'Determine alle positive integers $n>1$ with the following property:\nFor each colouring of the lattice points in the plane with $n$ colours, there are three lattice points of the same colour forming an isosceles right triangle with legs parallel to the coordinate axes.', 'solution': None, 'answer': None, 'problem_type': 'Combinatorics', 'question_type': 'math-word-problem', 'problem_is_valid': 'Yes', 'solution_is_valid': 'Yes', 'source': 'aops_forum', 'synthetic': False}
0.8913825750350952 {'problem': '4. Find all positive integers $n>1$, such that if all lattice points in the plane are colored with $n$ colors, there always exists a monochromatic isosceles triangle, with the two equal sides parallel to the coordinate axes.', 'solution': '4. The required is for all $n \\in \\mathbf{Z}_{+}, n>1$.\n\nStrengthened proposition: For each constant $c>0$ (only related to $n$), there exists a sufficiently large $M=M(c, n)$, such that for any $N \\in \\mathbf{Z}_{

The usefulness of our simple embedding search demonstrates the huge potential of semantic search. For all our problems we could identify exact or similar matches, which provide valuable insights into the necessary techniques to solve the current problem. And since we can cite the database, we don't have to deal with LLM hallucinations, which are especially common on such complex tasks.

(c) Mia Müßig